In [39]:
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Activation

In [40]:
import numpy as np
import cv2
import random as rn

In [41]:
def image_augmentation(img,h,v,r,z):
    col,row = img.shape
    #vertical and horizontal shift:
    matrix = np.float32( [[1,0,h],[0,1,v]] )
    img = cv2.warpAffine(img, matrix, (row,col))
    #rotation and zoom:
    matrix = cv2.getRotationMatrix2D((row/2,col/2), r, z)
    img = cv2.warpAffine(img, matrix, (row,col))
    return img

In [42]:
def image_resize(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(32,32))
    return img

In [43]:
def create_train_dataset(size):
    dataset=[]
    for i in range(1,size+1):
        dog=image_resize("D:\\Image_Dataset\\train\\dog\\dog."+str(i)+".jpg")
        cat=image_resize("D:\\Image_Dataset\\train\\cat\\cat."+str(i)+".jpg")
        dataset.append([dog, 1, 0])
        dataset.append([cat, 0, 1])
        '''for i in range(1):
            aug_dog=image_augmentation(dog,rn.randint(0,0),rn.randint(0,0),rn.choice([0,90,180,270]),rn.uniform(1.1,2))
            aug_cat=image_augmentation(cat,rn.randint(0,0),rn.randint(0,0),rn.choice([0,90,180,270]),rn.uniform(1.1,2))
            dataset.append([aug_dog,1,0])
            dataset.append([aug_cat,0,1])'''
    dataset=np.asarray(dataset)
    np.random.shuffle( dataset )
    op=dataset[:,1:3]

    x=dataset[:,0:1].ravel()
    ip=[]
    for i in range( np.size(x) ):
        ip.append(x[i])
    ip=np.asarray(ip).reshape(-1,32,32,1)
    ip=ip/255
    return ip,op

In [44]:
def CNN():
    model = Sequential()
    
    model.add( Conv2D(32,(3,3),input_shape=X.shape[1:]) )
    model.add( Activation('relu') )
    model.add( MaxPooling2D(pool_size=(2,2)) )
    
    model.add( Conv2D(32,(3,3)) )
    model.add( Activation('relu') )
    model.add( MaxPooling2D(pool_size=(2,2)) )
    
    model.add( Conv2D(64,(3,3)) )
    model.add( Activation('relu') )
    model.add( MaxPooling2D(pool_size=(2,2)) )
    
    model.add( Flatten() )
    model.add( Dense(64) )
    model.add( Dropout(0.5) )
    model.add( Activation('relu') )
    
    model.add( Dense(2) )
    model.add( Activation('sigmoid') )
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X,y, epochs=20, batch_size=20, validation_split=0.1)
    
    return model

In [45]:
X,y = create_train_dataset(12450)
model = CNN()

Train on 22410 samples, validate on 2490 samples
Epoch 1/20
22410/22410 [==============================] - 32s 1ms/step - loss: 0.6667 - acc: 0.5933 - val_loss: 0.6422 - val_acc: 0.6353
Epoch 2/20
22410/22410 [==============================] - 32s 1ms/step - loss: 0.5851 - acc: 0.6949 - val_loss: 0.5230 - val_acc: 0.7357
Epoch 3/20
22410/22410 [==============================] - 32s 1ms/step - loss: 0.5248 - acc: 0.7428 - val_loss: 0.5152 - val_acc: 0.7514
Epoch 4/20
22410/22410 [==============================] - 32s 1ms/step - loss: 0.4897 - acc: 0.7664 - val_loss: 0.4917 - val_acc: 0.7602
Epoch 5/20
22410/22410 [==============================] - 33s 1ms/step - loss: 0.4638 - acc: 0.7808 - val_loss: 0.4753 - val_acc: 0.7739
Epoch 6/20
22410/22410 [==============================] - 32s 1ms/step - loss: 0.4329 - acc: 0.8018 - val_loss: 0.4628 - val_acc: 0.7827
Epoch 7/20
22410/22410 [==============================] - 34s 2ms/step - loss: 0.4086 - acc: 0.8121 - val_loss: 0.4616 - val_acc:

In [46]:
model.save("MyCNN-3.h5")